# LangChain: Agents

AI Agents are autonomous programs that can take decisions by themselves, using an LLM as a reasoning engine!

Additionally, they can utilize a number of predefined tools, interact with external systems & APIs by structured prompts & responses.

They can plan, Act and reflect based on particular Agentic Architectures.

## Outline:

* Using built in LangChain tools: DuckDuckGo search and Wikipedia
* Defining your own tools

In [1]:
# Importing env variables.
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

import warnings
warnings.filterwarnings("ignore")

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

In [2]:
# Selecting the model!

# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo" # This in my case
else:
    llm_model = "gpt-3.5-turbo-0301"

## Built-in LangChain tools

There are multiple tools, and as this is an old version of `LangChain`, now it supports a lot more, especially using `LangGraph's` granular control & persistent memory layer!

Using some of the predefined tools below !

In [3]:
# Install
#!pip install -U wikipedia

In [4]:
# Importing common tools.
from langchain.agents.agent_toolkits import create_python_agent
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.chat_models import ChatOpenAI

In [5]:
# Instantiating OpenAI model with temp=0 (for better predictability)
llm = ChatOpenAI(temperature=0, model=llm_model)

In [6]:
# Loading few tools
# llm-math has a calculator, & wikipedia has an API connectivity for search!
tools = load_tools(["llm-math","wikipedia"], llm=llm)

In [7]:
# Creating the agent
# Using AgentType as CHAT (Chat effective), REACT (Prompting technique for Reason-Act approach)

agent= initialize_agent(
    tools, 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

In [8]:
# Let's ask few simple math problems (To use llm-math tool!)
agent("What is the 25% of 300?")



> Entering new AgentExecutor chain...
Thought: We can use the calculator tool to find 25% of 300.
Action:
```
{
  "action": "Calculator",
  "action_input": "25% of 300"
}
```
Observation: Answer: 75.0
Thought:Final Answer: 75.0

> Finished chain.


{'input': 'What is the 25% of 300?', 'output': '75.0'}

In [12]:
agent("I have 6 months (Jan-Jun 25') to prepare for placements,\
i can do deep work only 3 days in a row, then 2-3 days i work with 50% efficiency!\
How much will i work? (in days?)")



> Entering new AgentExecutor chain...
Question: How much will I work in days during the 6 months of preparation for placements?
Thought: To calculate the total number of days worked, we need to consider the pattern of deep work for 3 days followed by 2-3 days of work with 50% efficiency.
Action:
```
{
  "action": "Calculator",
  "action_input": "3 + (2.5 * 3)"
}
```

Observation: Answer: 10.5
Thought:I made a mistake in my previous calculation. Let's correct it and find out the total number of days worked during the 6 months of preparation for placements.
Action:
```
{
  "action": "Calculator",
  "action_input": "3 + (2 * 0.5) + 3 + (2 * 0.5) + 3 + (2 * 0.5) + 3 + (2 * 0.5) + 3 + (2 * 0.5) + 3 + (2 * 0.5)"
}
```
Observation: Answer: 24.0
Thought:Final Answer: During the 6 months of preparation for placements, you will work for a total of 24 days.

> Finished chain.


{'input': "I have 6 months (Jan-Jun 25') to prepare for placements,i can do deep work only 3 days in a row, then 2-3 days i work with 50% efficiency!How much will i work? (in days?)",
 'output': 'During the 6 months of preparation for placements, you will work for a total of 24 days.'}

## Wikipedia example

Let's ask something that needs **wikipedia**!

In [13]:
question = "Tom M. Mitchell is an American computer scientist \
and the Founders University Professor at Carnegie Mellon University (CMU)\
what book did he write?"
result = agent(question) 



> Entering new AgentExecutor chain...
Thought: I can use Wikipedia to find out which book Tom M. Mitchell wrote.
Action:
```
{
  "action": "Wikipedia",
  "action_input": "Tom M. Mitchell"
}
```
Observation: Page: Tom M. Mitchell
Summary: Tom Michael Mitchell (born August 9, 1951) is an American computer scientist and the Founders University Professor at Carnegie Mellon University (CMU). He is a founder and former chair of the Machine Learning Department at CMU. Mitchell is known for his contributions to the advancement of machine learning, artificial intelligence, and cognitive neuroscience and is the author of the textbook Machine Learning. He is a member of the United States National Academy of Engineering since 2010. He is also a Fellow of the American Academy of Arts and Sciences, the American Association for the Advancement of Science and a Fellow and past president of the Association for the Advancement of Artificial Intelligence. In October 2018, Mitchell was appointed as the 

In [14]:
# Let's ask more!
agent("Who was Rajneesh OSHO, write a factual summary about him!")



> Entering new AgentExecutor chain...
Question: Who was Rajneesh OSHO, write a factual summary about him!
Thought: I should use Wikipedia to gather information about Rajneesh OSHO.
Action:
```
{
  "action": "Wikipedia",
  "action_input": "Rajneesh OSHO"
}
```
Observation: Page: Rajneesh
Summary: Rajneesh (born Chandra Mohan Jain; 11 December 1931 – 19 January 1990), also known as Acharya Rajneesh, Bhagwan Shree Rajneesh, and later as Osho (Hindi pronunciation: [ˈo:ʃo:]), was an Indian godman, philosopher, mystic and founder of the Rajneesh movement. He was viewed as a controversial new religious movement leader during his life. He rejected institutional religions, insisting that spiritual experience could not be organized into any one system of religious dogma. As a guru, he advocated meditation and taught a unique form called dynamic meditation. Rejecting traditional ascetic practices, he advocated that his followers live fully in the world but without attachment to it. In expressin

{'input': 'Who was Rajneesh OSHO, write a factual summary about him!',
 'output': "Rajneesh OSHO, also known as Acharya Rajneesh and later as Osho, was an Indian godman, philosopher, mystic, and founder of the Rajneesh movement. He advocated meditation, dynamic meditation, and living fully in the world without attachment. Rajneesh's teachings have influenced Western New Age thought, and his ashram in Pune, India, now known as OSHO International Meditation Resort, continues to operate under the management of the Osho International Foundation."}

## Python Agent

Let's use a specialized python agent, that is equipped with a python repl.

In [15]:
# Create python Agent
agent = create_python_agent(
    llm,
    tool=PythonREPLTool(),
    verbose=True
)

In [16]:
# Test it
customer_list = [["Harrison", "Chase"], 
                 ["Lang", "Chain"],
                 ["Dolly", "Too"],
                 ["Elle", "Elem"], 
                 ["Geoff","Fusion"], 
                 ["Trance","Former"],
                 ["Jen","Ayai"]
                ]

In [17]:
agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""") 



> Entering new AgentExecutor chain...
We can use the sorted() function in Python to sort the list of customers based on last name first and then first name.
Action: Use the sorted() function
Action Input: sorted([['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']], key=lambda x: (x[1], x[0]))
Observation: Use the sorted() function is not a valid tool, try another one.
Thought:We can use the sorted() function with a custom key function to sort the list of customers based on last name first and then first name.
Action: Use the sorted() function with a custom key function
Action Input: sorted([['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']], key=lambda x: (x[1], x[0]))
Observation: Use the sorted() function with a custom key function is not a valid tool, try another one.
Thought:We can use the sorted() function wi

"[['Jen', 'Ayai'], ['Harrison', 'Chase'], ['Lang', 'Chain'], ['Elle', 'Elem'], ['Trance', 'Former'], ['Geoff', 'Fusion'], ['Dolly', 'Too']]"

In [18]:
# We can get more deeper insights on how this work!
# Using langchain.debug as True

langchain.debug = True

# Running agent
agent.run(f"""Concatenate the names in sub lists \
And return a list of full-names \
and print the output: {customer_list}""") 


NameError: name 'langchain' is not defined

#### View detailed outputs of the chains

In [19]:
# We can get more deeper insights on how this work!
# Using langchain.debug as True

import langchain
langchain.debug = True

# Running agent
agent(f"""Concatenate the names in sub lists \
And return a list of full-names \
and print the output: {customer_list}""") 


[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Concatenate the names in sub lists And return a list of full-names and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "Concatenate the names in sub lists And return a list of full-names and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: You are an agent designed to write and execute python code to answer questions.\nYou have access to a python REPL, which you can use 

[llm/end] [1:chain:AgentExecutor > 8:chain:LLMChain > 9:llm:ChatOpenAI] [878.087ms] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Action: Use a for loop to join the names in each sublist\nAction Input: full_names = []\nfor names in [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]:\n    full_names.append(' '.join(names))\nprint(full_names)",
        "generation_info": null,
        "message": {
          "content": "Action: Use a for loop to join the names in each sublist\nAction Input: full_names = []\nfor names in [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]:\n    full_names.append(' '.join(names))\nprint(full_names)",
          "additional_kwargs": {},
          "example": false
        }
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "prompt_tokens": 

[llm/end] [1:chain:AgentExecutor > 14:chain:LLMChain > 15:llm:ChatOpenAI] [919.62ms] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Action: Use a list comprehension to join the names in each sublist\nAction Input: full_names = [' '.join(names) for names in [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]]\nprint(full_names)",
        "generation_info": null,
        "message": {
          "content": "Action: Use a list comprehension to join the names in each sublist\nAction Input: full_names = [' '.join(names) for names in [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]]\nprint(full_names)",
          "additional_kwargs": {},
          "example": false
        }
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "prompt_tokens": 719,
      "completion_tokens": 8

[llm/end] [1:chain:AgentExecutor > 20:chain:LLMChain > 21:llm:ChatOpenAI] [756.21ms] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "I now know the final answer\nFinal Answer: ['Harrison Chase', 'Lang Chain', 'Dolly Too', 'Elle Elem', 'Geoff Fusion', 'Trance Former', 'Jen Ayai']",
        "generation_info": null,
        "message": {
          "content": "I now know the final answer\nFinal Answer: ['Harrison Chase', 'Lang Chain', 'Dolly Too', 'Elle Elem', 'Geoff Fusion', 'Trance Former', 'Jen Ayai']",
          "additional_kwargs": {},
          "example": false
        }
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "prompt_tokens": 950,
      "completion_tokens": 45,
      "total_tokens": 995
    },
    "model_name": "gpt-3.5-turbo"
  }
}
[chain/end] [1:chain:AgentExecutor > 20:chain:LLMChain] [756.6070000000001ms] Exiting Chain run with output:
{
  "text": "I now know the final answer\nFinal Answer: ['Harrison Chase', 'Lang Chain', 

{'input': "Concatenate the names in sub lists And return a list of full-names and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]",
 'output': "['Harrison Chase', 'Lang Chain', 'Dolly Too', 'Elle Elem', 'Geoff Fusion', 'Trance Former', 'Jen Ayai']"}

## Define your own tool

Defining our own custom tools as functions!

In [20]:
#!pip install DateTime

In [21]:
# Imports
from langchain.agents import tool
from datetime import date

In [22]:
# Tool to get a correct get!
@tool
def time(text: str) -> str:
    """Returns todays date, use this for any \
    questions related to knowing todays date. \
    The input should always be an empty string, \
    and this function will always return todays \
    date - any date mathmatics should occur \
    outside this function."""
    return str(date.today())

In [23]:
# Initialize the Agent with Time tool!!
agent= initialize_agent(
    tools + [time], 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

**Note**: 

The agent will sometimes come to the wrong conclusion (agents are a work in progress!). 

If it does, please try running it again.

In [24]:
# Let's run the agent!
try:
    result = agent("whats the date today?")
except: 
    print("exception on external access")

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "whats the date today?"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "whats the date today?",
  "agent_scratchpad": "",
  "stop": [
    "Observation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: Answer the following questions as best you can. You have access to the following tools:\n\nCalculator: Useful for when you need to answer questions about math.\nWikipedia: A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.\ntime: time(text: str) -> str - Returns todays date, use this for any     questions related to knowing todays date.     The input should always be an empty string,     and this function will always return to

Reminder: Download your notebook to you local computer to save your work.